# Lecture 02 - Interpreters

## Expressions

We represent simple expressions.

An expression can be
    - a number, e.g. ``Numb 3``
    - a binary operation, taking a first expression, an operator symbol defined by ``op``, and a second expression.

In [1]:
type op = Plus | Minus | Mult | Div 
type exp =  Numb of int | Op of exp * op * exp 

Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads



type op = Plus | Minus | Mult | Div


type exp = Numb of int | Op of exp * op * exp


For example, the following expression represents the expression ``3 + (2 + 5)``: 

In [2]:
let e = Op (Numb 3, Plus, Op (Numb 2, Plus, Numb 5))

val e : exp = Op (Numb 3, Plus, Op (Numb 2, Plus, Numb 5))


Here are some more examples of expressions: 

In [3]:
(* Representing the expression 5 *)
Numb 5;;

(* Representing the expression 3 + 8 *)
Op (Numb 3, Plus, Numb 8);; 

(* Representing the expression 2 * (3 + 8) *)
Op (Numb 2, Mult, Op (Numb 3, Plus, Numb 8));;

(* Representing the expressions (3 - 2)%0 *)
Op (Op (Numb 3, Minus, Numb 2), Div, Numb 0)

- : exp = Numb 5


- : exp = Op (Numb 3, Plus, Numb 8)


- : exp = Op (Numb 2, Mult, Op (Numb 3, Plus, Numb 8))


- : exp = Op (Op (Numb 3, Minus, Numb 2), Div, Numb 0)


We define an **interpreter**, ``eval : exp -> int`` that **evaluates** an expression to an ``int`` by traversing the expression. 

Note that we use a custom **exception**, that allows us to throw errors to inform the user of a problem we encountered. 
See the introduction of https://cs3110.github.io/textbook/chapters/data/exceptions.html to be reminded of the corresponding syntax.

Here, we define an exception which will allow us to throw a RuntimeError - and gives back information of type string.

In [4]:
exception RuntimeError of string

let rec eval (e : exp) : int = match e with 
    | Numb n -> n 
    | Op (e1, Plus, e2) -> eval e1 + eval e2
    | Op (e1, Minus, e2) -> eval e1 - eval e2 
    | Op (e1, Mult, e2) -> eval e1 * eval e2 
    | Op (e1, Div, e2) -> (match (eval e2) with 
                          | 0 -> raise (RuntimeError "Divison by 0")
                          | _ -> eval e1 / eval e2)

exception RuntimeError of string


val eval : exp -> int = <fun>


We **traverse** the syntax tree of ``e``. 
- If we encounter a number ``Numb n``, the result is simply the carried ``int``. 
- If we encounter the addition operation,  ``Op (e1, Plus, e2)``, we evaluate both expressions (the results will be of type ``int``) and add the two results together. 
- In the case of division we check that the second expression doesn't result in ``0``; otherwise, we throw a runtime error.

Here are some examples:

In [5]:
eval (Numb 5)

- : int = 5


In [6]:
eval (Op (Numb 3, Plus, Numb 8))

- : int = 11


In [7]:
eval (Op (Numb 2, Mult, Op (Numb 3, Plus, Numb 8)))

- : int = 22


In [8]:
eval (Op (Op (Numb 3, Minus, Numb 2), Div, Numb 0))

error: runtime_error

## Adding assignments (and contexts)

Next we add in assignments and statements. 

In [9]:
type op = Plus | Minus | Mult | Div 
type exp = Id of string (* NEW *)
          | Numb of int | Op of exp * op * exp 
type cmd = Asgn of string * exp
type program = P of cmd list * exp

type op = Plus | Minus | Mult | Div


type exp = Id of string | Numb of int | Op of exp * op * exp


type cmd = Asgn of string * exp


type program = P of cmd list * exp


Here is an example program: 

In [10]:
let p : program = 
P ([Asgn ("x", Numb 3); Asgn ("y", Numb 4)], Op (Id "x", Plus, Id "y"))

val p : program =
  P ([Asgn ("x", Numb 3); Asgn ("y", Numb 4)], Op (Id "x", Plus, Id "y"))


Remember that this means that we require an environment. 
Environments are dictionaries that map variable names to values. Whenever we require the value of a variable, we can look it up in the dictionary.

We use a pre-defined definition of a Map: 

(Don't worru about the specific details of the definition - all you need to do is use the functions Env.empty, Env.add, and Env.find.
If you want to know more about the details of modules, see the documentation: https://ocaml.org/docs/modules)

In [11]:
(* This will define maps with strings as key *)
module Env = Map.Make(String)

(* Env.empty denotes the empty environment. 
We can add elements to an environment via Env.add.
This is the environment which only binds “a” to 3. *)
let example_env = Env.add "a" 3 Env.empty;;

(* We can look up elements in an environment via Env.find.
Env.find throws an exception if the key does not exist.*)
Env.find "a" example_env;;

module Env :
  sig
    type key = String.t
    type 'a t = 'a Map.Make(String).t
    val empty : 'a t
    val is_empty : 'a t -> bool
    val mem : key -> 'a t -> bool
    val add : key -> 'a -> 'a t -> 'a t
    val update : key -> ('a option -> 'a option) -> 'a t -> 'a t
    val singleton : key -> 'a -> 'a t
    val remove : key -> 'a t -> 'a t
    val merge :
      (key -> 'a option -> 'b option -> 'c option) -> 'a t -> 'b t -> 'c t
    val union : (key -> 'a -> 'a -> 'a option) -> 'a t -> 'a t -> 'a t
    val compare : ('a -> 'a -> int) -> 'a t -> 'a t -> int
    val equal : ('a -> 'a -> bool) -> 'a t -> 'a t -> bool
    val iter : (key -> 'a -> unit) -> 'a t -> unit
    val fold : (key -> 'a -> 'b -> 'b) -> 'a t -> 'b -> 'b
    val for_all : (key -> 'a -> bool) -> 'a t -> bool
    val exists : (key -> 'a -> bool) -> 'a t -> bool
    val filter : (key -> 'a -> bool) -> 'a t -> 'a t
    val filter_map : (key -> 'a -> 'b option) -> 'a t -> 'b t
    val partition : (key -> 'a -> bool) 

val example_env : int Env.t = <abstr>


- : int = 3


Can you guess what is the result of the following expresssion? 

In [12]:
Env.empty;;

let env = Env.add "y" 3 Env.empty;;

let env' = Env.empty;;

Env.add "x" 5 env;;

Env.find "y" (Env.add "x" 5 (Env.add "y" 3 Env.empty)) 

- : 'a Env.t = <abstr>


val env : int Env.t = <abstr>


val env' : 'a Env.t = <abstr>


- : int Env.t = <abstr>


- : int = 3


That's enough to write an interpreter with assignments: 

In [13]:
(* Gets an expression and an environment, and yield an int *)
let rec eval (e : exp) env : int = match e with 
    | Id x -> Env.find x env
    | Numb n -> n 
    | Op (e1, Plus, e2) -> eval e1 env + eval e2 env
    | Op (e1, Minus, e2) -> eval e1 env - eval e2 env
    | Op (e1, Mult, e2) -> eval e1 env * eval e2 env
    | Op (e1, Div, e2) -> (match (eval e2 env) with 
                          | 0 -> raise (RuntimeError "Divison by 0")
                          | _ -> eval e1 env / eval e2 env)

(* Gets a command list and an environment, and yield a new environment *)
let rec eval_statements (cs : cmd list) env = match cs with 
    | [] -> env
    | Asgn (x, e) :: cs -> let env' = Env.add x (eval e env) env
                           in eval_statements cs env'

(* Gets a command list and an expression - evaluates to a new environment, and then 
evaluates the expression in this environment *)
let eval_program (P (cs, e)) = let env = eval_statements cs Env.empty 
                                 in eval e env


val eval : exp -> int Env.t -> int = <fun>


val eval_statements : cmd list -> int Env.t -> int Env.t = <fun>


val eval_program : program -> int = <fun>


Here's an example of an evaluation:

In [14]:
let example_program : program = P 
([Asgn ("x", Numb 3); Asgn ("y", Numb 4); Asgn ("x", Numb 5)], 
Op (Id "x", Plus, Id "y"));;

eval_program example_program

val example_program : program =
  P ([Asgn ("x", Numb 3); Asgn ("y", Numb 4); Asgn ("x", Numb 5)],
   Op (Id "x", Plus, Id "y"))


- : int = 9


## An Interpreter for SIMP

We now come to an interpreter for the whole of SIMP. 
Let's start with the reprsentation of SIMP as an abstract data type.

Note that because of the block (``Begin program``), commands require the definition of a program - and programs require the definition of commands. 
We also say that programs and commands are **mutually recursive**. 

In OCaml, this means that we have to define the two data types ``cmd`` and ``program`` at the same time. 
We combine the two via an ``and`` (the same is valid or the later functions.)

In [19]:
type op = Plus | Minus | Mult | Div 

type exp = Id of string | Numb of int | Op of exp * op * exp 

type cond = Eq | Neq | Lte | Lt | Gte | Gt 
type condexp = Cop of exp * cond * exp
                                          
type cmd = Asgn of string * exp 
         | Ite of condexp * cmd * cmd | If of condexp * cmd 
         | While of condexp * cmd
         | Begin of program 
         | Print of exp 
         
and program = Program of string list * cmd list

type op = Plus | Minus | Mult | Div


type exp = Id of string | Numb of int | Op of exp * op * exp


type cond = Eq | Neq | Lte | Lt | Gte | Gt


type condexp = Cop of exp * cond * exp


type cmd =
    Asgn of string * exp
  | Ite of condexp * cmd * cmd
  | If of condexp * cmd
  | While of condexp * cmd
  | Begin of program
  | Print of exp
and program = Program of string list * cmd list


We now come to the definition of evaluation for Simp. 

We evaluate the comparisons to ``1`` if true and ``0`` otherwise. 
This allows us to define evaluation for ``if`` and ``while``. 

In the case of printing, we use OCaml's predefined printing function ``print_endline``. (Please use only this function, as Jupyter notebooks might otherwise not print out what you expect.) 
We can concatenate strings via `^`.

In [20]:
let rec eval (e : exp) env : int = match e with 
    | Id x -> Env.find x env 
    | Numb n -> n 
    | Op (e1, Plus, e2) -> eval e1 env + eval e2 env
    | Op (e1, Minus, e2) -> eval e1 env - eval e2 env
    | Op (e1, Mult, e2) -> eval e1 env * eval e2 env
    | Op (e1, Div, e2) ->  (match (eval e2 env) with 
                          | 0 -> raise (RuntimeError "Divison by 0")
                          | _ -> eval e1 env / eval e2 env)
                          
let eval_condexp (e : condexp) env : int = match e with                          
    | Cop (e1, Eq, e2) -> if (eval e1 env = eval e2 env) then 1 else 0
    | Cop (e1, Neq, e2) -> if (eval e1 env = eval e2 env) then 0 else 1
    | Cop (e1, Lte, e2) -> if (eval e1 env <= eval e2 env) then 1 else 0
    | Cop (e1, Lt, e2) -> if (eval e1 env < eval e2 env) then 1 else 0
    | Cop (e1, Gte, e2) -> if (eval e1 env >= eval e2 env) then 1 else 0
    | Cop (e1, Gt, e2) -> if (eval e1 env > eval e2 env) then 1 else 0

let rec eval_cmd (c : cmd) env  = match c with 
    | Asgn (x, e) -> Env.add x (eval e env) env
    | If (e, c) -> (match (eval_condexp e env) with 
                        | 1 -> eval_cmd c env 
                        | 0 -> env
                        | _ -> raise (RuntimeError "Error in condition of If")
                        )
    | Ite (e, c1, c2) -> (match (eval_condexp e env) with 
                        | 1 -> eval_cmd c1 env 
                        | 0 -> eval_cmd c2 env
                        | _ -> raise (RuntimeError "Error in condition of If"))
    | While (e, c) -> (match (eval_condexp e env) with 
                        | 1 -> eval_cmd (While (e, c)) (eval_cmd c env)
                        | _ -> env)
    | Begin p -> let _ = eval_program p env in env
    | Print e -> let _ = print_endline ("OUTPUT:" ^ string_of_int (eval e env)) in env
    
and eval_program p env = match p with 
    | Program (xs, cmds) -> eval_commands cmds Env.empty

and eval_commands (cs : cmd list) env = match cs with 
    | [] -> env
    | c :: cs -> eval_commands cs (eval_cmd c env)

val eval : exp -> int Env.t -> int = <fun>


val eval_condexp : condexp -> int Env.t -> int = <fun>


val eval_cmd : cmd -> int Env.t -> int Env.t = <fun>
val eval_program : program -> int Env.t -> int Env.t = <fun>
val eval_commands : cmd list -> int Env.t -> int Env.t = <fun>


Here is an example:

In [21]:
let p = Program ([], [Asgn ("x", Numb 3); Asgn ("x", Numb 4); Print (Id "x")]);;

eval_program p Env.empty;;

val p : program =
  Program ([], [Asgn ("x", Numb 3); Asgn ("x", Numb 4); Print (Id "x")])


OUTPUT:4


- : int Env.t = <abstr>
